In [21]:
import pandas as pd
import requests
from tqdm import tqdm
import geopandas as gpd
from shapely.geometry import Point

def geocode_address(address, timeout=10):
    API_URL = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        "address": address,
        "key": "write here your google api",
        "language": "ru"
    }
    try:
        response = requests.get(API_URL, params=params, timeout=timeout)
        if response.status_code == 200:
            data = response.json()
            if data['results']:
                lat = data['results'][0]['geometry']['location']['lat']
                lng = data['results'][0]['geometry']['location']['lng']
                return {"lat": lat, "lng": lng}
        else:
            print(f"Failed to geocode address with status code {response.status_code}: {address}")
    except requests.Timeout:
        print(f"Request timed out for address: {address}")
    except requests.RequestException as e:
        print(f"Request exception for address {address}: {e}")
    return None

import pandas as pd
import requests
from tqdm import tqdm
import geopandas as gpd
from shapely.geometry import Point

# existing geocode_address function 

# Placeholder path for your Excel file
file_path = r'C:\Users\[user]\Downloads\yourfile.xls'
df = pd.read_excel(file_path)

# Initialize an empty GeoDataFrame with the additional attributes
gdf = gpd.GeoDataFrame(columns=[
    'address', 'metro', 'walking_time_to_metro', 'square_meters', 'city', 'cost','geometry'
])

# Loop through the DataFrame, geocoding each address
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    street_name = row['Улица']  # Assuming 'Улица' is your column name for addresses
    city = row['Город']
    full_address = f"{street_name}, {city}, Russia"
    result = geocode_address(full_address)

    if result:
        # Create a Shapely Point from the result coordinates
        point = Point(result['lng'], result['lat'])

        # Create a new DataFrame for the current row and concatenate it with the main GeoDataFrame
        new_row = gpd.GeoDataFrame({
            'address': [street_name],
            'metro': [row['Метро']],
            'walking_time_to_metro': [row['Кол-во минут пешком до метро']],
            'square_meters': [row['Площадь']],
            'city': [city],
            'cost':[row['Цена, млн руб.']],
            'geometry': [point]
        }, columns=gdf.columns)
        gdf = pd.concat([gdf, new_row], ignore_index=True)
    else:
        print(f"Failed to geocode address: {full_address}")

# Set the coordinate reference system (CRS) to WGS84
gdf.set_crs(epsg=4326, inplace=True)

# Export to GeoJSON
geojson_file_path = r'your file path like - D:\geojson\matched_buildings.geojson'
gdf.to_file(geojson_file_path, driver='GeoJSON')


100%|██████████| 91/91 [00:34<00:00,  2.65it/s]
